###  Contributeurs : BELRHALMIA WALID et Mohamed Ali Darghouth et Ghiles Sidi Saïd


Question 1: Dry run: Train a first-order HMM using the training data. This is basically what we did in lab sessions for POS-tagging. Compute the error rate (at the character level) and compare this results with the dummiest classifier that just do nothing. You can also compute the number of errors your model can correct and the number of errors your model creates.

Import des packages utiles 

In [1]:
import pickle
from numpy import array, ones, zeros, multiply
import numpy as np



UNK = "<unk>"  # token to map all out-of-vocabulary words (OOVs)
UNKid = 0      # index for UNK

Loading Train and Test DataSet

In [2]:
train10 =   pickle.load(open('/Users/belrhalmia/Documents/TC4/typos-data/train10.pkl', 'rb'))

train20 =   pickle.load(open('/Users/belrhalmia/Documents/TC4/typos-data/train20.pkl', 'rb'))

test10 =  pickle.load(open('/Users/belrhalmia/Documents/TC4/typos-data/test10.pkl', 'rb'))

test20 =  pickle.load(open('/Users/belrhalmia/Documents/TC4/typos-data/test20.pkl', 'rb'))

# Compter les lettres et les tags

In [3]:
class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None, smoothing_obs = 0.01):


            """
            Builds a Hidden Markov Model
            * state_list is the list of state symbols [q_0...q_(N-1)]
            * observation_list is the list of observation symbols [v_0...v_(M-1)]
            * transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            * observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            * initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print "HMM creating with: "
            
            self.N = len(state_list)       # number of states
            self.M = len(observation_list) # number of possible emissions
            print str(self.N)+" states"
            print str(self.M)+" observations"
            self.omega_Y = state_list
            self.omega_X = observation_list
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            self.make_indexes() # build indexes, i.e the mapping between token and int
            self.smoothing_obs = smoothing_obs 
 
        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities array"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
                
            print "States:\n", self.Y_index
            print "Observations:\n", self.X_index


        def get_observationIndices( self, observations ):
            """return observation indices, i.e 
            return [self.O_index[o] for o in observations]
            and deals with OOVs
            """
            indices = zeros( len(observations), int )
            k = 0
            for o in observations:
                if o in self.X_index:
                    indices[k] = self.X_index[o]
                else:
                    indices[k] = UNKid
                k += 1
            return indices
        
        """From one each word
        - extract the letters and correction of rache one
        - (letterObservation,LetterState)
        """
        def data2indices(self, word): 
    
            carracterObservation = list()
            carracterState  = list()
            for carracter in word:
                observation = carracter[0]
                state = carracter[1]
                if observation in self.X_index:
                    carracterObservation.append(self.X_index[observation])
                else:
                    carracterObservation.append(UNKid)
                carracterState.append(self.Y_index[state])
            return carracterObservation,carracterState
        
        def observation_estimation(self, cpairs):
            """ Build the observation distribution: 
                observation_proba is the observation probablility matrix
                [b_ki],  b_ki = Pr(X_t=v_k|Y_t=q_i)"""
            # fill with counts
            for pair in cpairs: 
                observation=pair[0]
                state=pair[1]
                cpt=cpairs[pair]
                k = 0 # for <unk>
                if observation in self.X_index: 
                    k=self.X_index[observation]
                i=self.Y_index[state]
                self.observation_proba[k,i]=cpt
            # normalize
            self.observation_proba=self.observation_proba+self.smoothing_obs
            self.observation_proba=self.observation_proba/self.observation_proba.sum(axis=0).reshape(1,self.N)
        
        def transition_estimation(self, ctrans):
            """ Build the transition distribution: 
                transition_proba is the transition matrix with : 
                [a_ij] a[i,j] = Pr(Y_(t+1)=q_i|Y_t=q_j)
            """
            # fill with counts
            for pair in ctrans:
                i=self.Y_index[pair[0]]
                j=self.Y_index[pair[1]]
                self.transition_proba[i,j]=ctrans[pair]
            # normalize
            self.transition_proba=self.transition_proba/self.transition_proba.sum(axis=0).reshape(1,self.N)
     
            
        def init_estimation(self, cinits):
            """Build the init. distribution"""
            # fill with counts
            for state in cinits:
                i=self.Y_index[state]
                self.initial_state_proba[i]=cinits[state]
            # normalize
            self.initial_state_proba=self.initial_state_proba/sum(self.initial_state_proba)
        
        def supervised_training(self, cpairs, ctrans,cinits):
            """ Train the HMM's parameters. This function wraps everything"""
            self.observation_estimation(cpairs)
            self.transition_estimation(ctrans)
            self.init_estimation(cinits)
            
        def viterbi( self, observation ):
            """Find the most probable state sequence
            """
            product = zeros( self.N, float )
            delta = zeros( self.N, float )
            max_proba = zeros( (len(observation), self.N), int )
            modified_delta = zeros( self.N, float ) # modifier le délta
            # pour la première lettre de l'observation de chaque mot on multiplie sa matrice de probabilité d'émission(de chaque état) par la matrice de probabilité d'états initiaux.
            delta = self.observation_proba[observation[0]] * self.initial_state_proba   
            for L in xrange(1, len(observation)):
                for K in range(self.N):
                    max_proba[L, K] = multiply( delta, self.transition_proba[:, K]).argmax()       
                    modified_delta[K] = multiply( delta, self.transition_proba[:, K])[multiply( delta, self.transition_proba[:, K]).argmax()] * self.observation_proba[observation[L], K] 
               
                pivot=modified_delta
                modified_delta=delta
                delta  = pivot
            best_path = [delta.argmax()]                        
            for r in max_proba[-1:0:-1]:
                best_path.append( r[best_path[-1]] )                 
            best_path.reverse()
            return best_path

In [4]:
def make_counts(train):
    
    """
    Build different count tables to train a HMM1. Each count table is a dictionnary.
    Returns:
    * c_observation: No correct Letter Count (Observation Letter)
    * c_state: Correct Letters  ( State Letters)
    * c_pairs: count of pairs (nncorrectedletr,correctedletr)
    * c_transitions: count bigram
    * c_inits: count of tag found in the first position
    """
        
    c_carracterObservation = dict()
    c_carracterState = dict()
    c_pairs= dict()
    c_transitions = dict()
    c_inits = dict()
    
    
    for word in train:
        # we use i because of the transition counts
        for i in range(len(word)):
            couple=word[i]
            observation = couple[0]
            state = couple[1]
            # carracter counts
            if observation in c_carracterObservation:
                c_carracterObservation[observation]=c_carracterObservation[observation]+1
            else:
                c_carracterObservation[observation]=1
            # carracter  counts
            if state in c_carracterState:
                c_carracterState[state] = c_carracterState[state]+1
            else:
                c_carracterState[state]=1
            # observation counts
            if couple in c_pairs:
                c_pairs[couple]=c_pairs[couple]+1
            else:
                c_pairs[couple]=1
            # i >  0 -> transition counts
            if i > 0:
                trans = (word[i-1][1],state)
                if trans in c_transitions:
                    c_transitions[trans]=c_transitions[trans]+1
                else:
                    c_transitions[trans]=1
            # i == 0 -> counts for initial states
            else:
                if state in c_inits:
                    c_inits[state]=c_inits[state]+1
                else:
                    c_inits[state]=1
                    
    return c_carracterObservation ,c_carracterState,c_pairs, c_transitions, c_inits

In [5]:
def make_vocab(c_carracterObservation, threshold):
    """ 
    return a vocabulary by thresholding word counts. 
    inputs: 
    * c_words : a dictionnary that maps word to its counts
    * threshold: count must be >= to the threshold to be included
    
    returns: 
    * a word list
    """
    voc = list()
    voc.append(UNK)
    for carracter in c_carracterObservation:
        if c_carracterObservation[carracter] >= threshold:
            voc.append(carracter)
    return voc

In [7]:
ccarractero,ccarracterS,cpairs,ctrans,cinits = make_counts(train10)
vocab = make_vocab(ccarractero,10)
hmm = HMM(state_list=ccarracterS.keys(), observation_list=vocab,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None)
hmm.supervised_training(cpairs, ctrans, cinits)
hmm.observation_estimation(cpairs)
hmm.transition_estimation(ctrans)
hmm.init_estimation(cinits)

HMM creating with: 
26 states
27 observations
States:
{'a': 0, 'c': 1, 'b': 2, 'e': 3, 'd': 4, 'g': 5, 'f': 6, 'i': 7, 'h': 8, 'k': 9, 'j': 10, 'm': 11, 'l': 12, 'o': 13, 'n': 14, 'q': 15, 'p': 16, 's': 17, 'r': 18, 'u': 19, 't': 20, 'w': 21, 'v': 22, 'y': 23, 'x': 24, 'z': 25}
Observations:
{'a': 1, 'c': 2, 'b': 3, 'e': 4, 'd': 5, 'g': 6, 'f': 7, 'i': 8, 'h': 9, 'k': 10, 'j': 11, 'm': 12, 'l': 13, 'o': 14, 'n': 15, 'q': 16, 'p': 17, 's': 18, 'r': 19, 'u': 20, 't': 21, 'w': 22, 'v': 23, 'y': 24, 'x': 25, 'z': 26, '<unk>': 0}


### Partie Test
Dans cette partie nous allons appliquer notre hmm sur les deux fichiers de test : test10 (qui contient 10% d'erreurs') et test20 (qui contient 20% d'erreurs')

In [8]:
import numpy as np
correct_carracter=0
total_carracter=0
error_before_viterbi = 0
correction_error=0
carracter_corrected=0
total_carracter=0

for word in test10:
    
    observations,states = hmm.data2indices(word)
    
    # meilleur séquence déterminer par l'algorithme de viterbi 
    
    meilleur_sequence = hmm.viterbi(observations)  
    
    # calcul du nombre total des caractères cela va nous aider lors du calcul de l'accuracy et de l'erreur
    
    total_carracter= total_carracter + len(states)
    
    #Nombre d'erreurs avant d'avoir appliqué viterbi
    
    error_before_viterbi += sum (np.array([-1+x  for x in observations]) != np.array(states))
    
    #Nombre de caractères corrects par viterbi
    
    correct_carracter = correct_carracter + sum(np.array( meilleur_sequence)==np.array(states))

    
    # dans cette condition on va voir les mots pour lesquels Viterbi s'est trompé (on compare la meilleure séquence avec l'état du mot).
    # pour les mots où viterbi s'est trompé on compare la meilleure séquence avec l'observation, et comme ça on va obtenir les caractères que viterbi à corrigé mais d'une manière fausse.
    
    
    if (np.array(meilleur_sequence) != np.array(states)).all():
        correction_error = correction_error +  sum(np.array ([-1+x  for x in observations]) != np.array(meilleur_sequence))
        
    
    # nombre d'erreurs corrigées par HMM  
    if (np.array(meilleur_sequence) == np.array(states)).all():
        carracter_corrected += sum(np.array ([-1+x  for x in observations]) != np.array(meilleur_sequence))
    
print 'Le nombre total de tous les caractéres dans le fichier de test : ' +str(total_carracter)
print "Le nombre d'erreurs avant viterbi : " + str(error_before_viterbi)    
print "Le nombre d'erreurs que le modèle crée : " + str(correction_error)
print "Le nombre d'erreurs que le modèle corrige : " + str(carracter_corrected)
print 'Le nombre de carractère correcte par viterbi :' +str(correct_carracter)
print "L'accuarcy du modèle est :"+str(correct_carracter*100.0/total_carracter ) + " %"
print "L'erreur du modèle est : "+str(100 - correct_carracter*100.0/total_carracter ) + " %"

Le nombre total de tous les caractéres dans le fichier de test : 7320
Le nombre d'erreurs avant viterbi : 745
Le nombre d'erreurs que le modèle crée : 6
Le nombre d'erreurs que le modèle corrige : 201
Le nombre de carractère correcte par viterbi :6777
L'accuarcy du modèle est :92.5819672131 %
L'erreur du modèle est : 7.41803278689 %


In [9]:
import numpy as np
correct_carracter=0
total_carracter=0
error_before_viterbi = 0
correction_error=0
carracter_corrected=0
total_carracter=0

for word in test20:
    
    observations,states = hmm.data2indices(word)
    
    # meilleur séquence déterminer par l'algorithme de viterbi 
    
    meilleur_sequence = hmm.viterbi(observations)  
    
    # calcul du nombre total des caractères cela va nous aider lors du calcul de l'accuracy et de l'erreur
    
    total_carracter= total_carracter + len(states)
    
    #Nombre d'erreurs avant d'avoir appliqué viterbi
    
    error_before_viterbi += sum (np.array([-1+x  for x in observations]) != np.array(states))
    
    #Nombre de caractères corrects par viterbi
    
    correct_carracter = correct_carracter + sum(np.array( meilleur_sequence)==np.array(states))

    
    # dans cette condition on va voir les mots pour lesquels Viterbi s'est trompé (on compare la meilleure séquence avec l'état du mot).
    # pour les mots où viterbi s'est trompé on compare la meilleure séquence avec l'observation, et comme ça on va obtenir les caractères que viterbi à corrigé mais d'une manière fausse.
    
    
    if (np.array(meilleur_sequence) != np.array(states)).all():
        correction_error = correction_error +  sum(np.array ([-1+x  for x in observations]) != np.array(meilleur_sequence))
        
    
    # nombre d'erreurs corrigées par HMM  
    if (np.array(meilleur_sequence) == np.array(states)).all():
        carracter_corrected += sum(np.array ([-1+x  for x in observations]) != np.array(meilleur_sequence))
    
print 'Le nombre total de tous les caractéres dans le fichier de test : ' +str(total_carracter)
print "Le nombre d'erreurs avant viterbi : " + str(error_before_viterbi)    
print "Le nombre d'erreurs que le modèle crée : " + str(correction_error)
print "Le nombre d'erreurs que le modèle corrige : " + str(carracter_corrected)
print 'Le nombre de carractère correcte par viterbi :' +str(correct_carracter)
print "L'accuarcy du modèle est :"+str(correct_carracter*100.0/total_carracter ) + " %"
print "L'erreur du modèle est : "+str(100 - correct_carracter*100.0/total_carracter ) + " %"

Le nombre total de tous les caractéres dans le fichier de test : 16691
Le nombre d'erreurs avant viterbi : 3239
Le nombre d'erreurs que le modèle crée : 26
Le nombre d'erreurs que le modèle corrige : 655
Le nombre de carractère correcte par viterbi :14433
L'accuarcy du modèle est :86.4717512432 %
L'erreur du modèle est : 13.5282487568 %
